In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving abalone.data.csv to abalone.data.csv


In [ ]:
import io
import pandas as pd

# Assuming 'uploaded' is a dictionary containing the uploaded file
df = pd.read_csv(io.BytesIO(uploaded['abalone.data.csv']))

# Display the first few rows of the dataframe to ensure it was loaded correctly
print(df.head())


  Sex  Length  Diameter  Height  Whole_weight  Shucked_weight  Viscera_weight  \
0   M   0.455     0.365   0.095        0.5140          0.2245          0.1010   
1   M   0.350     0.265   0.090        0.2255          0.0995          0.0485   
2   F   0.530     0.420   0.135        0.6770          0.2565          0.1415   
3   M   0.440     0.365   0.125        0.5160          0.2155          0.1140   
4   I   0.330     0.255   0.080        0.2050          0.0895          0.0395   

   Shell_weight  Rings  
0         0.150     15  
1         0.070      7  
2         0.210      9  
3         0.155     10  
4         0.055      7  


In [ ]:
# Check the column names of the dataframe
print(df.columns)


Index(['Sex', 'Length', 'Diameter', 'Height', 'Whole_weight', 'Shucked_weight',
       'Viscera_weight', 'Shell_weight', 'Rings'],
      dtype='object')


In [ ]:
# Perform one-hot encoding for the 'Sex' column
df = pd.get_dummies(df, columns=['Sex'])

# Standardize the continuous features
continuous_features = ['Length', 'Diameter', 'Height', 'Whole_weight', 'Shucked_weight', 'Viscera_weight', 'Shell_weight']
scaler = StandardScaler()
df[continuous_features] = scaler.fit_transform(df[continuous_features])

# Display the first few rows of the dataset
print(df.head())


     Length  Diameter    Height  Whole_weight  Shucked_weight  Viscera_weight  \
0 -0.574558 -0.432149 -1.064424     -0.641898       -0.607685       -0.726212   
1 -1.448986 -1.439929 -1.183978     -1.230277       -1.170910       -1.205221   
2  0.050033  0.122130 -0.107991     -0.309469       -0.463500       -0.356690   
3 -0.699476 -0.432149 -0.347099     -0.637819       -0.648238       -0.607600   
4 -1.615544 -1.540707 -1.423087     -1.272086       -1.215968       -1.287337   

   Shell_weight  Rings  Sex_F  Sex_I  Sex_M  
0     -0.638217     15  False  False   True  
1     -1.212987      7  False  False   True  
2     -0.207139      9   True  False  False  
3     -0.602294     10  False  False   True  
4     -1.320757      7  False   True  False  


In [ ]:
X = df.drop('Rings', axis=1)
y = df['Rings']
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)# Convert boolean columns to numeric (0 and 1)
X_train['Sex_F'] = X_train['Sex_F'].astype(int)
X_train['Sex_I'] = X_train['Sex_I'].astype(int)
X_train['Sex_M'] = X_train['Sex_M'].astype(int)

X_test['Sex_F'] = X_test['Sex_F'].astype(int)
X_test['Sex_I'] = X_test['Sex_I'].astype(int)
X_test['Sex_M'] = X_test['Sex_M'].astype(int)

# Convert the datasets into PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)


In [ ]:
# Define the neural network class
class AbaloneNN(nn.Module):
    def __init__(self):
        super(AbaloneNN, self).__init__()
        self.fc1 = nn.Linear(10, 64)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

In [ ]:
import torch.optim as optim  # Import optim module for SGD optimizer
# Create an instance of the model
model = AbaloneNN()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
# Train the model with gradient clipping
def train_model(model, criterion, optimizer, X_train, y_train, epochs=500, clip_value=1.0):
    for epoch in range(epochs):
        # Forward pass
        outputs = model(X_train)
        loss = criterion(outputs, y_train)

        # Backward pass and optimization with gradient clipping
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip_value)
        optimizer.step()

        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Redefine the model and optimizer
model = AbaloneNN()
optimizer = optim.SGD(model.parameters(), lr=0.01)  # Reduced learning rate

# Train the model with gradient clipping
train_model(model, criterion, optimizer, X_train_tensor, y_train_tensor)


Epoch [10/500], Loss: 108.9513
Epoch [20/500], Loss: 105.8993
Epoch [30/500], Loss: 102.8419
Epoch [40/500], Loss: 99.7233
Epoch [50/500], Loss: 96.4825
Epoch [60/500], Loss: 93.0736
Epoch [70/500], Loss: 89.4663
Epoch [80/500], Loss: 85.6401
Epoch [90/500], Loss: 81.5857
Epoch [100/500], Loss: 77.3049
Epoch [110/500], Loss: 72.8095
Epoch [120/500], Loss: 68.1243
Epoch [130/500], Loss: 63.2849
Epoch [140/500], Loss: 58.3364
Epoch [150/500], Loss: 53.3347
Epoch [160/500], Loss: 48.3463
Epoch [170/500], Loss: 43.4479
Epoch [180/500], Loss: 38.7261
Epoch [190/500], Loss: 34.2747
Epoch [200/500], Loss: 30.1900
Epoch [210/500], Loss: 26.5607
Epoch [220/500], Loss: 23.4511
Epoch [230/500], Loss: 20.8741
Epoch [240/500], Loss: 18.7715
Epoch [250/500], Loss: 17.0275
Epoch [260/500], Loss: 15.5314
Epoch [270/500], Loss: 14.2127
Epoch [280/500], Loss: 13.0235
Epoch [290/500], Loss: 11.9311
Epoch [300/500], Loss: 10.9223
Epoch [310/500], Loss: 9.9960
Epoch [320/500], Loss: 9.1557
Epoch [330/500],

In [ ]:
def evaluate(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        mse = nn.MSELoss()
        loss = mse(outputs, y_test)
    return loss.item()

test_loss = evaluate(model, X_test_tensor, y_test_tensor)
print(f'Test Loss: {test_loss:.4f}')


Test Loss: 4.7404


In [ ]:
def evaluate_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        loss = criterion(outputs, y_test)
    return loss.item()

learning_rates = [0.001, 0.01, 0.1, 1.0]
results = []

for lr in learning_rates:
    model = AbaloneNN()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    train_model(model, criterion, optimizer, X_train_tensor, y_train_tensor)
    test_loss = evaluate_model(model, X_test_tensor, y_test_tensor)
    results.append((lr, test_loss))

# Tabulate the results
print("Learning Rate\tMSE")
for lr, mse in results:
    print(f"{lr}\t\t{mse:.4f}")


Epoch [10/500], Loss: 109.3999
Epoch [20/500], Loss: 109.1029
Epoch [30/500], Loss: 108.8052
Epoch [40/500], Loss: 108.5066
Epoch [50/500], Loss: 108.2072
Epoch [60/500], Loss: 107.9070
Epoch [70/500], Loss: 107.6058
Epoch [80/500], Loss: 107.3037
Epoch [90/500], Loss: 107.0005
Epoch [100/500], Loss: 106.6963
Epoch [110/500], Loss: 106.3910
Epoch [120/500], Loss: 106.0844
Epoch [130/500], Loss: 105.7766
Epoch [140/500], Loss: 105.4676
Epoch [150/500], Loss: 105.1573
Epoch [160/500], Loss: 104.8458
Epoch [170/500], Loss: 104.5330
Epoch [180/500], Loss: 104.2188
Epoch [190/500], Loss: 103.9033
Epoch [200/500], Loss: 103.5864
Epoch [210/500], Loss: 103.2680
Epoch [220/500], Loss: 102.9482
Epoch [230/500], Loss: 102.6268
Epoch [240/500], Loss: 102.3038
Epoch [250/500], Loss: 101.9792
Epoch [260/500], Loss: 101.6529
Epoch [270/500], Loss: 101.3251
Epoch [280/500], Loss: 100.9954
Epoch [290/500], Loss: 100.6640
Epoch [300/500], Loss: 100.3308
Epoch [310/500], Loss: 99.9958
Epoch [320/500], L

In [ ]:
# Using Adagrad optimizer
optimizer_adagrad = torch.optim.Adagrad(model.parameters(), lr=0.1)
train_model(model, criterion, optimizer_adagrad, X_train_tensor, y_train_tensor)
test_loss_adagrad = evaluate_model(model, X_test_tensor, y_test_tensor)
print(f'Test Loss with Adagrad: {test_loss_adagrad:.4f}')


Epoch [10/500], Loss: 4.3315
Epoch [20/500], Loss: 4.2924
Epoch [30/500], Loss: 4.1209
Epoch [40/500], Loss: 3.9559
Epoch [50/500], Loss: 3.8911
Epoch [60/500], Loss: 3.8435
Epoch [70/500], Loss: 3.8766
Epoch [80/500], Loss: 3.7878
Epoch [90/500], Loss: 3.7307
Epoch [100/500], Loss: 3.6911
Epoch [110/500], Loss: 3.6614
Epoch [120/500], Loss: 3.6336
Epoch [130/500], Loss: 3.6426
Epoch [140/500], Loss: 3.5843
Epoch [150/500], Loss: 3.5613
Epoch [160/500], Loss: 3.5282
Epoch [170/500], Loss: 3.5027
Epoch [180/500], Loss: 3.4840
Epoch [190/500], Loss: 3.4642
Epoch [200/500], Loss: 3.4594
Epoch [210/500], Loss: 3.4212
Epoch [220/500], Loss: 3.3945
Epoch [230/500], Loss: 3.3868
Epoch [240/500], Loss: 3.3469
Epoch [250/500], Loss: 3.3280
Epoch [260/500], Loss: 3.2929
Epoch [270/500], Loss: 3.2769
Epoch [280/500], Loss: 3.2655
Epoch [290/500], Loss: 3.2446
Epoch [300/500], Loss: 3.2286
Epoch [310/500], Loss: 3.2219
Epoch [320/500], Loss: 3.2055
Epoch [330/500], Loss: 3.1824
Epoch [340/500], Lo

In [ ]:
class AbaloneNN(nn.Module):
    def __init__(self):
        super(AbaloneNN, self).__init__()
        self.fc1 = nn.Linear(10, 64)
        self.sigmoid = nn.Sigmoid()
        self.fc2 = nn.Linear(64, 32)
        self.sigmoid2 = nn.Sigmoid()
        self.fc3 = nn.Linear(32, 16)
        self.sigmoid3 = nn.Sigmoid()
        self.fc4 = nn.Linear(16, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.sigmoid(x)
        x = self.fc2(x)
        x = self.sigmoid2(x)
        x = self.fc3(x)
        x = self.sigmoid3(x)
        x = self.fc4(x)
        return x

model_sigmoid = AbaloneNN()
optimizer_sgd = torch.optim.SGD(model_sigmoid.parameters(), lr=0.1)
train_model(model_sigmoid, criterion, optimizer_sgd, X_train_tensor, y_train_tensor)
test_loss_sigmoid = evaluate_model(model_sigmoid, X_test_tensor, y_test_tensor)
print(f'Test Loss with Sigmoid Activation: {test_loss_sigmoid:.4f}')


Epoch [10/500], Loss: 73.6814
Epoch [20/500], Loss: 39.5185
Epoch [30/500], Loss: 16.0889
Epoch [40/500], Loss: 10.2789
Epoch [50/500], Loss: 10.0570
Epoch [60/500], Loss: 9.6797
Epoch [70/500], Loss: 8.9910
Epoch [80/500], Loss: 7.9843
Epoch [90/500], Loss: 7.2122
Epoch [100/500], Loss: 6.9333
Epoch [110/500], Loss: 6.8101
Epoch [120/500], Loss: 6.7170
Epoch [130/500], Loss: 6.6342
Epoch [140/500], Loss: 6.5558
Epoch [150/500], Loss: 6.4778
Epoch [160/500], Loss: 6.3967
Epoch [170/500], Loss: 6.3092
Epoch [180/500], Loss: 6.2121
Epoch [190/500], Loss: 6.1024
Epoch [200/500], Loss: 5.9791
Epoch [210/500], Loss: 5.8442
Epoch [220/500], Loss: 5.7045
Epoch [230/500], Loss: 5.5698
Epoch [240/500], Loss: 5.4489
Epoch [250/500], Loss: 5.3468
Epoch [260/500], Loss: 5.2645
Epoch [270/500], Loss: 5.2001
Epoch [280/500], Loss: 5.1506
Epoch [290/500], Loss: 5.1126
Epoch [300/500], Loss: 5.0835
Epoch [310/500], Loss: 5.0607
Epoch [320/500], Loss: 5.0427
Epoch [330/500], Loss: 5.0280
Epoch [340/500

In [ ]:
class AbaloneNN(nn.Module):
    def __init__(self, num_hidden_layers=10):
        super(AbaloneNN, self).__init__()
        self.num_hidden_layers = num_hidden_layers
        self.layers = nn.ModuleList()

        # Input layer
        self.layers.append(nn.Linear(10, 64))
        self.layers.append(nn.Sigmoid())

        # Hidden layers
        for _ in range(num_hidden_layers):
            self.layers.append(nn.Linear(64, 64))
            self.layers.append(nn.Sigmoid())

        # Output layer
        self.layers.append(nn.Linear(64, 1))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

# Create an instance of the model with 10 hidden layers
model_10_layers = AbaloneNN(num_hidden_layers=10)

# Train the model
optimizer_sgd = torch.optim.SGD(model_10_layers.parameters(), lr=0.01)
train_model(model_10_layers, criterion, optimizer_sgd, X_train_tensor, y_train_tensor)

# Evaluate the model
test_loss_10_layers = evaluate_model(model_10_layers, X_test_tensor, y_test_tensor)
print(f'Test Loss with 10 Hidden Layers: {test_loss_10_layers:.4f}')


Epoch [10/500], Loss: 98.5437
Epoch [20/500], Loss: 90.8813
Epoch [30/500], Loss: 83.5500
Epoch [40/500], Loss: 76.5450
Epoch [50/500], Loss: 69.8633
Epoch [60/500], Loss: 63.5032
Epoch [70/500], Loss: 57.4649
Epoch [80/500], Loss: 51.7501
Epoch [90/500], Loss: 46.3622
Epoch [100/500], Loss: 41.3060
Epoch [110/500], Loss: 36.5882
Epoch [120/500], Loss: 32.2167
Epoch [130/500], Loss: 28.2010
Epoch [140/500], Loss: 24.5520
Epoch [150/500], Loss: 21.2819
Epoch [160/500], Loss: 18.4042
Epoch [170/500], Loss: 15.9339
Epoch [180/500], Loss: 13.8869
Epoch [190/500], Loss: 12.2804
Epoch [200/500], Loss: 11.1325
Epoch [210/500], Loss: 10.4626
Epoch [220/500], Loss: 10.2840
Epoch [230/500], Loss: 10.2840
Epoch [240/500], Loss: 10.2840
Epoch [250/500], Loss: 10.2840
Epoch [260/500], Loss: 10.2840
Epoch [270/500], Loss: 10.2840
Epoch [280/500], Loss: 10.2840
Epoch [290/500], Loss: 10.2840
Epoch [300/500], Loss: 10.2840
Epoch [310/500], Loss: 10.2840
Epoch [320/500], Loss: 10.2840
Epoch [330/500], 

In [ ]:
# Create an instance of the model with 15 hidden layers
model_15_layers = AbaloneNN(num_hidden_layers=15)

# Train the model
optimizer_sgd = torch.optim.SGD(model_15_layers.parameters(), lr=0.01)
train_model(model_15_layers, criterion, optimizer_sgd, X_train_tensor, y_train_tensor)

# Evaluate the model
test_loss_15_layers = evaluate_model(model_15_layers, X_test_tensor, y_test_tensor)
print(f'Test Loss with 15 Hidden Layers: {test_loss_15_layers:.4f}')


Epoch [10/500], Loss: 108.2062
Epoch [20/500], Loss: 100.1259
Epoch [30/500], Loss: 92.3855
Epoch [40/500], Loss: 84.9778
Epoch [50/500], Loss: 77.8974
Epoch [60/500], Loss: 71.1406
Epoch [70/500], Loss: 64.7053
Epoch [80/500], Loss: 58.5916
Epoch [90/500], Loss: 52.8008
Epoch [100/500], Loss: 47.3363
Epoch [110/500], Loss: 42.2030
Epoch [120/500], Loss: 37.4075
Epoch [130/500], Loss: 32.9581
Epoch [140/500], Loss: 28.8643
Epoch [150/500], Loss: 25.1373
Epoch [160/500], Loss: 21.7897
Epoch [170/500], Loss: 18.8352
Epoch [180/500], Loss: 16.2889
Epoch [190/500], Loss: 14.1670
Epoch [200/500], Loss: 12.4868
Epoch [210/500], Loss: 11.2666
Epoch [220/500], Loss: 10.5257
Epoch [230/500], Loss: 10.2844
Epoch [240/500], Loss: 10.2840
Epoch [250/500], Loss: 10.2840
Epoch [260/500], Loss: 10.2840
Epoch [270/500], Loss: 10.2840
Epoch [280/500], Loss: 10.2840
Epoch [290/500], Loss: 10.2840
Epoch [300/500], Loss: 10.2840
Epoch [310/500], Loss: 10.2840
Epoch [320/500], Loss: 10.2840
Epoch [330/500]